In [19]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/pauli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk import word_tokenize
from nltk import sent_tokenize

In [21]:
from utils.json_utils import read_json, write_json

In [22]:
coreferenced_docs = read_json("8_coreference_resolution_black_clover.json")

In [23]:
non_coreferenced_docs = read_json("1_scrapping_black_clover.json")

In [24]:
empty_docs = []
for i in range(0, len(coreferenced_docs)):
    if len(coreferenced_docs[i]) == 0:
        empty_docs.append(i)

In [25]:
for i in sorted(empty_docs,reverse=True):
    coreferenced_docs.pop(i)
    non_coreferenced_docs.pop(i)

In [26]:
tokenized_docs = list()
for doc in coreferenced_docs:
    doc_sentences = sent_tokenize(doc)
    tokenized_docs.append([])
    for sentence in doc_sentences:
        tokenized_docs[-1].append([])
        words = word_tokenize(sentence)
        for word in words:
            tokenized_docs[-1][-1].append(word)

In [27]:
doc_lengths = []
empty_docs = []
i = 0
for doc in tokenized_docs:
    token_count = 0
    for sentence in doc:
        token_count += len(sentence)
    if token_count == 0:
        empty_docs.append(i)
    doc_lengths.append(token_count)
    i += 1
sorted(doc_lengths)

[0,
 0,
 0,
 26,
 28,
 36,
 38,
 68,
 100,
 129,
 190,
 200,
 222,
 225,
 228,
 247,
 258,
 293,
 294,
 310,
 329,
 339,
 342,
 345,
 358,
 385,
 386,
 397,
 398,
 401,
 404,
 416,
 421,
 426,
 447,
 447,
 456,
 458,
 459,
 471,
 471,
 471,
 485,
 493,
 494,
 505,
 518,
 533,
 539,
 545,
 558,
 593,
 596,
 598,
 599,
 605,
 611,
 624,
 631,
 644,
 644,
 655,
 658,
 684,
 689,
 710,
 711,
 715,
 741,
 743,
 757,
 763,
 764,
 767,
 771,
 787,
 787,
 791,
 800,
 820,
 824,
 833,
 841,
 841,
 877,
 891,
 911,
 911,
 942,
 954,
 958,
 963,
 963,
 966,
 967,
 985,
 1001,
 1018,
 1035,
 1035,
 1038,
 1039,
 1065,
 1120,
 1134,
 1141,
 1149,
 1151,
 1161,
 1167,
 1172,
 1177,
 1179,
 1221,
 1243,
 1246,
 1285,
 1311,
 1318,
 1371,
 1396,
 1407,
 1427,
 1458,
 1498,
 1504,
 1529,
 1553,
 1563,
 1597,
 1704,
 1717,
 1767]

In [28]:
for i in sorted(empty_docs, reverse=True):
    coreferenced_docs.pop(i)
    tokenized_docs.pop(i)
    non_coreferenced_docs.pop(i)

In [29]:
tagged_tokenized_docs = [[nltk.pos_tag(tokenized_sentence) for tokenized_sentence in tokenized_sentences]for tokenized_sentences in tokenized_docs]

In [30]:
not2lemmatize_tagged_tokenized_docs = [[[(tagged_token[0], tagged_token[1], tagged_token[1]) for tagged_token in tagged_tokenized_sentence] for tagged_tokenized_sentence in tagged_tokenized_sentences] for tagged_tokenized_sentences in tagged_tokenized_docs ]

In [31]:
characters = read_json('3_characters_black_clover.json')

In [32]:
def get_subjects_in_sent(tokenized_sent):
    return set(map(lambda x : x[0], filter(lambda x: x[2] == 'NNP' and x[0] in characters, tokenized_sent)))

In [33]:
aux = list()
for tagged_tokenized_sentences in not2lemmatize_tagged_tokenized_docs:
    aux.append([])
    for tagged_tokenized_sentence in tagged_tokenized_sentences:
        subjects = list(get_subjects_in_sent(tagged_tokenized_sentence))
        aux[-1].append({
            "subjects": subjects, 
            "tokens" : []
        })
        for tagged_token in tagged_tokenized_sentence:
            aux[-1][-1]["tokens"].append(tagged_token[0])
nonLemmatizedWsubj_docs = aux
nonLemmatizedWsubj_docs

[[{'subjects': [],
   'tokens': ['In',
    'Hage',
    ',',
    'a',
    'priest',
    'finds',
    'two',
    'babies',
    'abandoned',
    'outside',
    'a',
    'priest',
    'church',
    '.']},
  {'subjects': ['Asta', 'Yuno'],
   'tokens': ['a',
    'priest',
    'takes',
    'two',
    'babies',
    'abandoned',
    'outside',
    'his',
    'church',
    'inside',
    'and',
    'discovers',
    'two',
    'babies',
    'abandoned',
    'outside',
    'his',
    'church',
    'names',
    'to',
    'be',
    'Yuno',
    'and',
    'Asta',
    '.']},
  {'subjects': ['Asta', 'Lily'],
   'tokens': ['Fifteen',
    'years',
    'later',
    ',',
    'Asta',
    'proposes',
    'to',
    'Sister',
    'Lily',
    ',',
    'who',
    'refuses',
    'repeatedly',
    '.']},
  {'subjects': ['Asta', 'Yuno'],
   'tokens': ['Yuno',
    'and',
    'the',
    'other',
    'orphans',
    'criticize',
    'Asta',
    'and',
    'point',
    'out',
    'Yuno',
    'lack',
    'of',
    'magic'

In [34]:
aux_coref = list()
aux_non_coref = list()
aux_bert_coref = list()
aux_bert_non_coref = list()
i = 0
for nonLemmatizedWsubj_sentences in nonLemmatizedWsubj_docs:
    
    subjects = set()
    token_count = 0
    for nonLemmatizedWsubj_sentence in nonLemmatizedWsubj_sentences:
        subjects.update(nonLemmatizedWsubj_sentence["subjects"])
        token_count += len(nonLemmatizedWsubj_sentence["tokens"])
    if token_count < 300:
        aux_coref.append({
            "subjects" : list(subjects),
            "doc" : re.sub(r" [ ]+", " ", coreferenced_docs[i].replace('\n', ' ').strip())
        })
        aux_bert_coref.append({
            "subjects" : list(subjects),
            "doc" : re.sub(r" [ ]+", " ", coreferenced_docs[i].replace('\n', ' ').strip())
        })
        aux_non_coref.append({
            "subjects" : list(subjects),
            "doc" : re.sub(r" [ ]+", " ", non_coreferenced_docs[i].replace('\n', ' ').strip())
        })
        aux_bert_non_coref.append({
            "subjects" : list(subjects),
            "doc" : re.sub(r" [ ]+", " ", non_coreferenced_docs[i].replace('\n', ' ').strip())
        })
    elif token_count < 600:
        aux_coref.append({
            "subjects" : list(subjects),
            "doc" : re.sub(r" [ ]+", " ", coreferenced_docs[i].replace('\n', ' ').strip())
        })
        aux_non_coref.append({
            "subjects" : list(subjects),
            "doc" : re.sub(r" [ ]+", " ", non_coreferenced_docs[i].replace('\n', ' ').strip())
        })
    i += 1
non_coref_nonTokenizedWsubj_docs = aux_non_coref
non_coref_nonTokenizedWsubj_docs_bert = aux_bert_non_coref
nonTokenizedWsubj_docs = aux_coref
nonTokenizedWsubj_docs_bert = aux_bert_coref

In [35]:
print(len(nonTokenizedWsubj_docs))
print(len(nonTokenizedWsubj_docs_bert))

52
16


In [36]:
write_json(nonTokenizedWsubj_docs, "18_docs_with_subjects.json")
write_json(nonTokenizedWsubj_docs_bert, "18_docs_with_subjects_bert.json")
write_json(non_coref_nonTokenizedWsubj_docs, "18_docs_with_subjects_non_coref.json")
write_json(non_coref_nonTokenizedWsubj_docs_bert, "18_docs_with_subjects_non_coref_bert.json")